In [1]:
# Uncertainty Quantification for Brain Tumor Classification
# Methods: Softmax, MC Dropout, Deep Ensemble, EDL, TTDA

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
# from sklearn.metrics import accuracy_score
import pandas as pd

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
base_dir = "../data/"

In [ ]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # adjust if images are RGB
])

# Paths
base_dir = "../data/"
train_folder = "../data/Training"
test_folder = "../data/Testing"

# Load datasets
train_ds = datasets.ImageFolder(root=train_folder, transform=transform)
test_ds = datasets.ImageFolder(root=test_folder, transform=transform)

# Data loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

# Class names
class_names = train_ds.classes
print("Classes:", class_names)

Classes: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [16]:
class BrainTumorCNN(nn.Module):
    def __init__(self, dropout_p=0.5):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2), nn.Dropout(dropout_p),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2), nn.Dropout(dropout_p),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2), nn.Dropout(dropout_p)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512), nn.ReLU(), nn.Dropout(dropout_p),
            nn.Linear(512, 4)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [17]:
def train_model(model, train_loader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for _ in range(epochs):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = criterion(model(x), y)
            loss.backward()
            optimizer.step()
    return model

In [18]:
# --- Evaluation Metrics ---
def compute_entropy(probs):
    return -np.sum(probs * np.log(probs + 1e-8), axis=1)

def compute_ece(probs, labels, n_bins=15):
    bin_boundaries = np.linspace(0, 1, n_bins + 1)
    confidences = np.max(probs, axis=1)
    predictions = np.argmax(probs, axis=1)
    ece = 0.0
    for i in range(n_bins):
        bin_lower, bin_upper = bin_boundaries[i], bin_boundaries[i+1]
        in_bin = (confidences >= bin_lower) & (confidences < bin_upper)
        if np.any(in_bin):
            acc = np.mean(predictions[in_bin] == labels[in_bin])
            avg_conf = np.mean(confidences[in_bin])
            ece += np.abs(avg_conf - acc) * np.mean(in_bin)
    return ece

In [19]:
def evaluate_model(name, probs, labels):
    preds = np.argmax(probs, axis=1)
    acc = (preds == labels).sum() / len(labels)
    entropy = np.mean(compute_entropy(probs))
    ece = compute_ece(probs, labels)
    return {"Method": name, "Accuracy": f"{acc*100:.2f}%", "Avg. Entropy ↓": f"{entropy:.2f}", "ECE ↓": f"{ece:.2f}"}

In [20]:
def predict(model, loader):
    model.eval()
    probs, labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = F.softmax(model(x), dim=1).cpu().numpy()
            probs.append(out)
            labels.extend(y.numpy())
    return np.vstack(probs), np.array(labels)

In [59]:
# --- Prediction Wrappers ---
def predict(model, loader):
    model.eval()
    probs, labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            out = F.softmax(model(x), dim=1).cpu().numpy()
            probs.append(out)
            labels.extend(y.numpy())
    return np.vstack(probs), np.array(labels)

def predict_mc_dropout(model, loader, T=50):
    model.train()
    probs_mc = []
    for _ in range(T):
        probs, _ = predict(model, loader)
        probs_mc.append(probs)
    return np.mean(np.stack(probs_mc), axis=0)

def predict_ttda(model, loader, T=50):
    aug = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    dataset_aug = datasets.ImageFolder(os.path.join(base_dir, "Testing"), transform=aug)
    loader_aug = DataLoader(dataset_aug, batch_size=32, shuffle=False)
    return predict_mc_dropout(model, loader_aug, T)

def predict_ensemble(models, loader):
    all_probs = [predict(m, loader)[0] for m in models]
    return np.mean(np.stack(all_probs), axis=0)

In [22]:
results = []

In [ ]:
base_model = train_model(BrainTumorCNN(), train_loader)
probs, labels = predict(base_model, test_loader)
results.append(evaluate_model("Softmax", probs, labels))

In [62]:
mc_probs = predict_mc_dropout(base_model, test_loader)
results.append(evaluate_model("MC Dropout", mc_probs, labels))

In [60]:
ttda_probs = predict_ttda(base_model, test_loader)
results.append(evaluate_model("TTDA", ttda_probs, labels))

In [33]:
ensemble_models = []
for _ in range(3):
    m = BrainTumorCNN().to(device)
    optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)
    m.train()
    for epoch in range(5):
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            loss = nn.CrossEntropyLoss()(m(x), y)
            loss.backward()
            optimizer.step()
    ensemble_models.append(m)
ens_probs = predict_ensemble(ensemble_models, test_loader)
results.append(evaluate_model("Deep Ensemble", ens_probs, labels))

In [34]:
results_df = pd.DataFrame(results).set_index("Method")
print(results_df)

                   Accuracy Avg. Entropy ↓ ECE ↓
Method                                          
Softmax (Baseline)   85.51%           0.37  0.01
MC Dropout           85.51%           0.37  0.01
TTDA (Optional)      82.61%           0.44  0.01
Deep Ensemble        88.02%           0.46  0.06


In [35]:
# --- Evidential Deep Learning ---
def relu_evidence(logits): return F.relu(logits)

def KL_divergence(alpha):
    K = alpha.shape[1]
    beta = torch.ones((1, K)).to(alpha.device)
    S_alpha = torch.sum(alpha, dim=1, keepdim=True)
    S_beta = torch.sum(beta, dim=1, keepdim=True)
    lnB = torch.lgamma(S_alpha) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True)
    lnB_uni = torch.sum(torch.lgamma(beta), dim=1, keepdim=True) - torch.lgamma(S_beta)
    dg0 = torch.digamma(S_alpha)
    dg1 = torch.digamma(alpha)
    kl = torch.sum((alpha - beta) * (dg1 - dg0), dim=1, keepdim=True)
    return (kl + lnB + lnB_uni).mean()

def edl_mse_loss(p, alpha, global_step, annealing_step):
    S = torch.sum(alpha, dim=1, keepdim=True)
    label = F.one_hot(p, num_classes=alpha.size(1)).float()
    A = torch.sum((label - (alpha / S)) ** 2, dim=1, keepdim=True)
    B = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)
    annealing_coef = min(1.0, global_step / annealing_step)
    return (A + B).mean() + annealing_coef * KL_divergence(alpha)

In [36]:
# --- EDL Inference ---
class BrainTumorEDL(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = BrainTumorCNN().features
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512), nn.ReLU(),
            nn.Linear(512, 4)  # no softmax
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

def predict_edl(model, loader):
    model.eval()
    probs, labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            logits = model(x)
            evidence = relu_evidence(logits)
            alpha = evidence + 1
            prob = alpha / torch.sum(alpha, dim=1, keepdim=True)
            probs.append(prob.cpu().numpy())
            labels.extend(y.numpy())
    return np.vstack(probs), np.array(labels)

In [54]:
edl_model = BrainTumorEDL().to(device)
optimizer = torch.optim.Adam(edl_model.parameters(), lr=1e-3)
edl_model.train()
global_step = 0
for epoch in range(10):
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        logits = edl_model(x)
        evidence = relu_evidence(logits)
        alpha = evidence + 1
        loss = edl_mse_loss(y, alpha, global_step, annealing_step=100)
        loss.backward()
        optimizer.step()
        global_step += 1
edl_probs, edl_labels = predict_edl(edl_model, test_loader)
results = results[:4]
results.append(evaluate_model("EDL", edl_probs, edl_labels))

In [43]:
results[:4]

[{'Method': 'Softmax (Baseline)',
  'Accuracy': '85.51%',
  'Avg. Entropy ↓': '0.37',
  'ECE ↓': '0.01'},
 {'Method': 'MC Dropout',
  'Accuracy': '85.51%',
  'Avg. Entropy ↓': '0.37',
  'ECE ↓': '0.01'},
 {'Method': 'TTDA (Optional)',
  'Accuracy': '82.61%',
  'Avg. Entropy ↓': '0.44',
  'ECE ↓': '0.01'},
 {'Method': 'Deep Ensemble',
  'Accuracy': '88.02%',
  'Avg. Entropy ↓': '0.46',
  'ECE ↓': '0.06'}]

In [63]:
results_df_2 = pd.DataFrame(results).set_index("Method")
print(results_df_2)

                   Accuracy Avg. Entropy ↓ ECE ↓
Method                                          
Softmax (Baseline)   85.51%           0.37  0.01
MC Dropout           85.51%           0.37  0.01
TTDA (Optional)      82.61%           0.44  0.01
Deep Ensemble        88.02%           0.46  0.06
EDL (Proposed)       92.68%           1.38  0.63
TTDA                 82.61%           0.44  0.02
TTDA                 82.68%           0.44  0.02
MC Dropout           85.51%           0.37  0.01


In [65]:
# Save Softmax 
torch.save(base_model.state_dict(), "../models/softmax/softmax_model.pth")

# Save EDL model
torch.save(edl_model.state_dict(), "../models/edl/edl_model.pth")

# Save Deep Ensemble (save all models)
for i, model in enumerate(ensemble_models):
    torch.save(model.state_dict(), f"../models/ensemble/ensemble_model_{i}.pth")